In [1]:
import pandas as pd
import numpy as np
import os 
from utils.dataframe_tools import filter_out_nan 

In [45]:
df = pd.read_csv('./Data/Test/merge_tls_test_01.csv') 
df_block = pd.read_csv('./Data/Test/merge_tls_test_01/discrete/0.csv')

In [46]:
list_col = df.columns.tolist()
list_col_block = df_block.columns.tolist()

In [9]:
def find_fields_by_prefix(layers: list, layer_prefixes: dict, fields: list, len_prefix: int, init: bool): 
    next_layers = []
    if len(layers) > 0: 
        if init: 
            for field in fields: 
                prefix = field.split('.')[0] 
                if prefix not in layers and len(field.split('.')) - len_prefix == 0: 
                    layer_prefixes['statistics'].append(field) 
                elif field.startswith(prefix) and len(field.split('.')) - len_prefix == 1: 
                    layer_prefixes[prefix].append(field) 
                    next_layers.append(field) 
                # for tls, it does not have field like tls.xx, only has tls.xx.xx
        else: 
            for prefix in layers: 
                # for prefix in layer_prefixes[layer]: 
                for field in fields: 
                    if field.startswith(prefix) and len(field.split('.')) - len_prefix == 1: 
                        if prefix not in layer_prefixes: 
                            layer_prefixes[prefix] = [] 
                            layer_prefixes[prefix].append(field) 
                        else: 
                            layer_prefixes[prefix].append(field) 
                    else: 
                        continue 
                    next_layers.append(field) 
            # len_prefix += 1 
    init = False
    return layer_prefixes, next_layers, init 

In [24]:
def find_fields_by_prefix(layers: list, layer_prefixes: dict, fields: list, init: bool): 
    next_layers = []
    if len(layers) > 0: 
        if init: 
            for field in fields: 
                prefix = field.split('.')[0] 
                len_prefix = len(prefix.split('.')) 
                if prefix not in layers and len(field.split('.')) - len_prefix == 0: 
                    layer_prefixes['statistics'].append(field) 
                elif field.startswith(prefix) and len(field.split('.')) - len_prefix == 1: 
                    layer_prefixes[prefix].append(field) 
                    next_layers.append(field) 
                # for tls, it does not have field like tls.xx, only has tls.xx.xx 
            empty_layers = [] 
            extra_len_prefix = len_prefix
            # check up the empty layers in initial, and increase the length of prefix. 
            # We hold an assumption that the emptiness means the length of prefix is not long enough
            for key in layer_prefixes.keys(): 
                if len(layer_prefixes[key]) == 0: 
                    empty_layers.append(key) 
            while len(empty_layers) > 0: 
                extra_len_prefix += 1
                for layer in empty_layers[:]: 
                    for field in fields: 
                        if field.startswith(layer): 
                            prefix = '.'.join(field.split('.')[:extra_len_prefix])
                            if field.startswith(prefix) and len(field.split('.')) - extra_len_prefix == 1: 
                                layer_prefixes[layer].append(field) 
                                next_layers.append(field) 
                    empty_layers.remove(layer) 
                    
        else: 
            for prefix in layers: 
                len_prefix = len(prefix.split('.')) 
                # for prefix in layer_prefixes[layer]: 
                for field in fields: 
                    if field.startswith(prefix) and len(field.split('.')) - len_prefix == 1: 
                        if prefix not in layer_prefixes: 
                            layer_prefixes[prefix] = [] 
                            layer_prefixes[prefix].append(field) 
                        else: 
                            layer_prefixes[prefix].append(field) 
                    else: 
                        continue 
                    next_layers.append(field) 
            # len_prefix += 1 
    init = False
    return layer_prefixes, next_layers, init 

In [47]:
def find_fields_by_prefix_logically(layers: list, layer_prefixes: dict, fields: list, len_prefix: int): 
    next_layers = [] 
    if len(layers) > 0: 
        for field in fields: 
            prefix = '.'.join(field.split('.')[:len_prefix]) 
            if prefix not in layers and len(field.split('.')) - len_prefix == 0: 
                layer_prefixes['statistics'].append(field) 
            elif field.startswith(prefix) and len(field.split('.')) - len_prefix == 1: # 
                if prefix not in layer_prefixes: 
                    layer_prefixes[prefix] = [] 
                layer_prefixes[prefix].append(field) 
                next_layers.append(field) 
            elif field.startswith(prefix) and len(field.split('.')) - len_prefix >= 2: 
                current_layer = '.'.join(field.split('.')[:len_prefix + 1]) 
                if prefix in layer_prefixes: 
                    if current_layer not in layer_prefixes[prefix]: 
                        layer_prefixes[prefix].append(current_layer) 
                        next_layers.append(current_layer) 
                else: # in order to handle the single structure fields like 'tls.ber.bitstring.padding'
                    layer_prefixes[prefix] = [] 
                    layer_prefixes[prefix].append(current_layer) 
                    next_layers.append(current_layer) 
    return layer_prefixes, next_layers 

def protocol_tree(list_fields: list, list_layers = ['eth', 'ip', 'tcp', 'tls']): 
    """
    Find the hierarchy structure of protocols by handling the csv columns. 
    """
    dict_protocol_tree = {item: [] for item in list_layers} 
    dict_protocol_tree['statistics'] = [] 
    lens = [len(item.split('.')) for item in list_fields]
    len_prefix = 1 # length of current prefix, i.e. eth 
    max_field_len = max(lens)
    # init = True
    while len_prefix < max_field_len: 
        dict_protocol_tree, list_layers = find_fields_by_prefix_logically(list_layers, dict_protocol_tree, list_fields, len_prefix) 
        len_prefix += 1
    return dict_protocol_tree 

In [42]:
def protocol_tree(list_fields: list, list_layers = ['eth', 'ip', 'tcp', 'tls']): 
    """
    Find the hierarchy structure of protocols by handling the csv columns. 
    """
    dict_protocol_tree = {item: [] for item in list_layers} 
    dict_protocol_tree['statistics'] = [] 
    lens = [len(item.split('.')) for item in list_fields]
    len_prefix = 1 # length of current prefix, i.e. eth 
    max_field_len = max(lens)
    init = True
    while len_prefix < max_field_len: 
        dict_protocol_tree, list_layers, init = find_fields_by_prefix(list_layers, dict_protocol_tree, list_fields, init) 
        len_prefix += 1
    return dict_protocol_tree 

In [48]:
dict_tree = protocol_tree(list_col_block) 
print(dict_tree)

{'eth': ['eth.dst', 'eth.src', 'eth.type'], 'ip': ['ip.version', 'ip.hdr_len', 'ip.dsfield', 'ip.len', 'ip.id', 'ip.flags', 'ip.frag_offset', 'ip.ttl', 'ip.proto', 'ip.checksum', 'ip.src', 'ip.dst'], 'tcp': ['tcp.srcport', 'tcp.dstport', 'tcp.stream', 'tcp.len', 'tcp.seq', 'tcp.seq_raw', 'tcp.ack', 'tcp.ack_raw', 'tcp.hdr_len', 'tcp.flags', 'tcp.window_size_value', 'tcp.window_size', 'tcp.checksum', 'tcp.urgent_pointer'], 'tls': [], 'statistics': ['frame_num', 'reassembled_segments'], 'eth.dst': ['eth.dst.lg', 'eth.dst.ig'], 'eth.src': ['eth.src.lg', 'eth.src.ig'], 'ip.dsfield': ['ip.dsfield.dscp', 'ip.dsfield.ecn'], 'ip.flags': ['ip.flags.rb', 'ip.flags.df', 'ip.flags.mf'], 'tcp.flags': ['tcp.flags.res', 'tcp.flags.ae', 'tcp.flags.cwr', 'tcp.flags.ece', 'tcp.flags.urg', 'tcp.flags.ack', 'tcp.flags.push', 'tcp.flags.reset', 'tcp.flags.syn', 'tcp.flags.fin', 'tcp.flags.str']}


In [ ]:
from graphviz import Digraph

def create_protocol_tree_graph(data, output_file='protocol_tree'):
    # Initialize a directed graph
    dot = Digraph(comment='Protocol Tree', format='png')
    dot.attr(rankdir='TB')  # Top-to-bottom layout
    dot.attr('node', shape='ellipse')  # Default node shape
    dot.attr('edge', arrowhead='vee')

    # Root node
    root_id = 'root'
    dot.node(root_id, 'Protocols', shape='box', style='filled', fillcolor='lightblue')

    # Function to add nodes and edges recursively
    def add_nodes(key, parent_id, prefix=''):
        # Create a unique node ID
        node_id = f"{parent_id}_{key.replace('.', '_')}"
        label = key.split('.')[-1] if '.' in key else key  # Show only the last part for brevity
        dot.node(node_id, label)

        # Add edge from parent to this node
        dot.edge(parent_id, node_id)

        # If key has children (fields or nested keys)
        if key in data:
            for field in data[key]:
                # Determine if field is a nested key
                field_parts = field.split('.')
                field_key = '.'.join(field_parts[:len(prefix.split('.')) + 1 if prefix else 1])
                
                if field_key in data:
                    # Nested key, recurse
                    add_nodes(field_key, node_id, field)
                else:
                    # Leaf node (field)
                    field_id = f"{node_id}_{field.replace('.', '_')}"
                    field_label = field.split('.')[-1]  # Show only the last part
                    dot.node(field_id, field_label, shape='plaintext')
                    dot.edge(node_id, field_id)

    # Add top-level keys (eth, ip, tcp, tls, statistics)
    for key in ['eth', 'ip', 'tcp', 'tls', 'statistics']:
        if key in data:
            add_nodes(key, root_id)

    # Render and save the graph
    dot.render(output_file, view=True, cleanup=True)
    return dot 

create_protocol_tree_graph(dict_tree, 'protocol_tree')

In [38]:
from treelib import Node, Tree


def create_protocol_tree_text(data):
    # Initialize a tree
    tree = Tree()
    tree.create_node("Protocols", "root")  # Root node

    # Function to add nodes recursively
    def add_nodes(key, parent_id, prefix=''):
        # Add the current key as a node
        node_id = f"{parent_id}_{key.replace('.', '_')}"
        label = key.split('.')[-1] if '.' in key else key  # Show only the last part
        tree.create_node(label, node_id, parent=parent_id)

        # If key has children
        if key in data:
            for field in data[key]:
                # Determine if field is a nested key
                field_parts = field.split('.')
                field_key = '.'.join(field_parts[:len(prefix.split('.')) + 1 if prefix else 1])
                
                if field_key in data:
                    # Nested key, recurse
                    add_nodes(field_key, node_id, field)
                else:
                    # Leaf node (field)
                    field_id = f"{node_id}_{field.replace('.', '_')}"
                    field_label = field.split('.')[-1]  # Show only the last part
                    tree.create_node(field_label, field_id, parent=node_id)

    # Add top-level keys
    for key in ['eth', 'ip', 'tcp', 'tls', 'statistics']:
        if key in data:
            add_nodes(key, "root")

    # Display the tree
    tree.show()
    return tree

create_protocol_tree_text(dict_tree)

DuplicatedNodeIdError: Can't create node with ID 'root_eth_eth'

In [7]:
test = 'frame_num' 
test1 = 'eth.dst.ig' 
test2 = 'eth.dst'
a = test.split('.') 
b = test1.split('.') 
c = test2.split('.')
print(a) 
print(b)
print(c)
print(c in b)

['frame_num']
['eth', 'dst', 'ig']
['eth', 'dst']
False


In [ ]:
def protocol_tree_extract(list_fields: list, list_layers = ['eth', 'ip', 'tcp', 'tls']): 
    dict_fields = {} 
    for layer in list_layers: 
        for field in list_fields: 

    for field in list_fields: 
        s_field = field.split('.') 
        if len(s_field) == 1 and s_field[0] not in list_layers: # statistical information, like frame_num, reassembled_info 
            dict_fields['statistics'].append(s_field) 
            
        if s_field[0] in list_layers: # layer prefix 
            if s_field[0] not in dict_fields.keys(): 
                last_len = len(s_field) 
                dict_fields[s_field[0]].append(s_field) 
            

    return 

In [3]:
all_nan_cols = df.columns[df.isna().all()] 

In [6]:
df_nonan = df.drop(columns=all_nan_cols)

In [2]:
path_test_directory = './Data/Test/filter_nan' 
# path_tls_csv = os.path.join(path_test_directory, 'tls_test_01.pcapng.csv') 
# path_tls_reassemble_csv = os.path.join(path_test_directory, 'reassemble_tls_test_01.pcapng.csv') 
path_test_csv = os.path.join(path_test_directory, 'filter_nan_merge_tls_test_01.csv') 

In [3]:
df = pd.read_csv(path_test_csv, index_col='frame_num') 
df.shape 

(1959, 190)

In [4]:
mask_df = df.notnull() 

In [21]:
dict_true = {} 
for col_num in range(mask_df.shape[1]): 
    list_true_indices = list(mask_df[mask_df.iloc[:, col_num]].index) 
    dict_true[col_num] = list_true_indices  

# for col in mask_df.columns: 
#     list_true_indices = list(mask_df[mask_df.loc[:, col]].index) 
#     dict_true[col] = list_true_indices 

In [22]:
# continuous
dict_block = {
    'block': [], 
    'columns': [], 
    'rows': [] 
} 

block_flag = 0 
last_key = 0
last_value = dict_true[last_key] 
list_col = []
for key, value in dict_true.items(): 
    if key == last_key: # init 
        dict_block['block'].append(block_flag) 
        list_col.append(key) 
        dict_block['columns'].append(list_col.copy()) 
        dict_block['rows'].append(dict_true[key]) 
    else: 
        if value == last_value: 
            dict_block['columns'][block_flag].append(key) 
        if value != last_value: 
            block_flag += 1 
            dict_block['block'].append(block_flag) 
            list_col.clear() 
            list_col.append(key) 
            dict_block['columns'].append(list_col.copy()) 
            dict_block['rows'].append(dict_true[key]) 
            last_key = key 
            last_value =value 


In [7]:
df_block = pd.DataFrame(dict_block)

In [23]:
from tqdm import tqdm 

block_values = dict_block['block'] 
columns_values = dict_block['columns'] 
rows_values = dict_block['rows'] 

list_sub_df = []

for block_name, columns, rows in tqdm(zip(block_values, columns_values, rows_values)): 
    subset_rows = df.loc[rows]
    sub_df = subset_rows.iloc[:, columns]
    list_sub_df.append(sub_df) 

41it [00:00, 826.97it/s]


In [7]:
dict_block = {
    'block': [], 
    'columns': [], 
    'rows': [] 
} 

block_flag = 0 
list_col = [] 
list_record_col = []
for key, value in dict_true.items(): 
    if key not in list_record_col: 
        for ik, iv in dict_true.items(): 
            if iv == value: 
                if block_flag not in dict_block['block']: 
                    dict_block['block'].append(block_flag) 
                    dict_block['rows'].append(dict_true[key]) 
                list_col.append(ik) 
            if iv != value: 
                continue 
        dict_block['columns'].append(list_col.copy()) 
        list_record_col.extend(list_col.copy()) 
        list_col.clear()
        block_flag += 1 

In [ ]:
print(dict_block)

In [8]:
df_block = pd.DataFrame(dict_block)

In [9]:
true_indices = list(mask_df[mask_df['tls_handshake_server_point_len']].index) 

In [6]:
intesection_series = mask_df.all()